In [1]:
import pandas as pd

In [2]:
the_office_df = pd.read_csv("output/the_office.csv")
the_office_df

,Unnamed: 0,tconst_series,tconst_episode,primaryTitle_series,primaryTitle_episode,seasonNumber,episodeNumber,averageRating,cast,characters
0,0,tt0386676,tt0664521,The Office,Pilot,1,1,7.6,"['Steve Carell', 'Rainn Wilson', 'John Krasins...","['Michael Scott', 'Dwight Schrute', 'Jim Halpe..."
1,1,tt0386676,tt0664514,The Office,Diversity Day,1,2,8.3,"['Steve Carell', 'Rainn Wilson', 'John Krasins...","['Michael Scott', 'Dwight Schrute', 'Jim Halpe..."
2,2,tt0386676,tt0664517,The Office,Health Care,1,3,7.9,"['Steve Carell', 'Rainn Wilson', 'John Krasins...","['Michael Scott', 'Dwight Schrute', 'Jim Halpe..."
3,3,tt0386676,tt0664523,The Office,The Alliance,1,4,8.1,"['Steve Carell', 'Rainn Wilson', 'John Krasins...","['Michael Scott', 'Dwight Schrute', 'Jim Halpe..."
4,4,tt0386676,tt0664510,The Office,Basketball,1,5,8.4,"['Steve Carell', 'Rainn Wilson', 'John Krasins...","['Michael Scott', 'Dwight Schrute', 'Jim Halpe..."
5,5,tt0386676,tt0664518,The Office,Hot Girl,1,6,7.8,"['Steve Carell', 'Rainn Wilson', 'John Krasins...","['Michael Scott', 'Dwight Schrute', 'Jim Halpe..."


In [3]:
working_df = pd.DataFrame()
index = pd.MultiIndex.from_product([list(the_office_df.primaryTitle_episode), ['Cast','Characters']],
                                  names=['Episode',''])
for idx, row in the_office_df.iterrows():
    
    # Turn cast members string from an episode into a list of individual cast members
    cast_list = row.cast[1:-1]
    cast_list = cast_list.replace('\'', '').split(', ')
    
    # Turn characters string from an episode into a list of individual characters
    character_list = row.characters[1:-1]
    character_list = character_list.replace('\'', '').split(', ')
    
    # Add the cast/character pairs to the working DataFrame
    working_df[row.primaryTitle_episode + ' Cast'] = cast_list
    working_df[row.primaryTitle_episode + ' Character'] = character_list
    #working_df[row.primaryTitle_episode] = list(zip(cast_list, character_list))
    
# Transpose the working_df so every row is associated to an episode of the show
working_df = working_df.T
working_df = pd.DataFrame(working_df.values, index=index, columns=range(1,15))

# Fix weirdly formatted names from the DataFrame
working_df.loc[('Basketball', 'Cast'), 8] = 'Patrice O\'Neal'
working_df.loc[('The Alliance', 'Characters'), 13] = 'Oscar Martinez'

working_df

1               2               3   \
Episode                                                                   
Pilot         Cast         Steve Carell    Rainn Wilson  John Krasinski   
              Characters  Michael Scott  Dwight Schrute     Jim Halpert   
Diversity Day Cast         Steve Carell    Rainn Wilson  John Krasinski   
              Characters  Michael Scott  Dwight Schrute     Jim Halpert   
Health Care   Cast         Steve Carell    Rainn Wilson  John Krasinski   
              Characters  Michael Scott  Dwight Schrute     Jim Halpert   
The Alliance  Cast         Steve Carell    Rainn Wilson  John Krasinski   
              Characters  Michael Scott  Dwight Schrute     Jim Halpert   
Basketball    Cast         Steve Carell    Rainn Wilson  John Krasinski   
              Characters  Michael Scott  Dwight Schrute     Jim Halpert   
Hot Girl      Cast         Steve Carell    Rainn Wilson  John Krasinski   
              Characters  Michael Scott  Dwight Schrute     Jim Halpert   

                                     4            5                   6   \
Episode                                                                    
Pilot         Cast        Jenna Fischer   B.J. Novak       Melora Hardin   
              Characters     Pam Beesly  Ryan Howard  Jan Levinson-Gould   
Diversity Day Cast        Jenna Fischer   B.J. Novak       Larry Wilmore   
              Characters     Pam Beesly  Ryan Howard           Mr. Brown   
Health Care   Cast        Jenna Fischer   B.J. Novak       Melora Hardin   
              Characters     Pam Beesly  Ryan Howard  Jan Levinson-Gould   
The Alliance  Cast        Jenna Fischer   B.J. Novak        David Denman   
              Characters     Pam Beesly  Ryan Howard        Roy Anderson   
Basketball    Cast        Jenna Fischer   B.J. Novak        David Denman   
              Characters     Pam Beesly  Ryan Howard        Roy Anderson   
Hot Girl      Cast        Jenna Fischer   B.J. Novak           Amy Adams   
              Characters     Pam Beesly  Ryan Howard                Katy   

                                          7                   8   \
Episode                                                            
Pilot         Cast              David Denman  Leslie David Baker   
              Characters        Roy Anderson      Stanley Hudson   
Diversity Day Cast        Leslie David Baker   Brian Baumgartner   
              Characters      Stanley Hudson        Kevin Malone   
Health Care   Cast        Leslie David Baker   Brian Baumgartner   
              Characters      Stanley Hudson        Kevin Malone   
The Alliance  Cast            Craig Robinson  Leslie David Baker   
              Characters      Darryl Philbin      Stanley Hudson   
Basketball    Cast            Craig Robinson      Patrice O'Neal   
              Characters      Darryl Philbin               Lonny   
Hot Girl      Cast             Melora Hardin        David Denman   
              Characters  Jan Levinson-Gould        Roy Anderson   

                                          9                  10  \
Episode                                                           
Pilot         Cast         Brian Baumgartner      Angela Kinsey   
              Characters        Kevin Malone      Angela Martin   
Diversity Day Cast             Kate Flannery       Mindy Kaling   
              Characters     Meredith Palmer       Kelly Kapoor   
Health Care   Cast             Kate Flannery   Charlie Hartsock   
              Characters     Meredith Palmer       Travel Agent   
The Alliance  Cast         Brian Baumgartner      Kate Flannery   
              Characters        Kevin Malone    Meredith Palmer   
Basketball    Cast        Leslie David Baker  Brian Baumgartner   
              Characters      Stanley Hudson       Kevin Malone   
Hot Girl      Cast        Leslie David Baker  Brian Baumgartner   
              Characters      Stanley Hudson       Kevin Malone   

                             

In [4]:
working_df.loc[('Basketball', 'Cast'), 1]

'Steve Carell'

In [25]:
# Find all the unique actors/actresses involved with the first season of The Office
cast_set = set()
for episode in working_df.index.get_level_values('Episode'):
    episode_set = set(working_df.loc[(episode,'Cast'),:])
    cast_set = cast_set.union(episode_set)

# Create a new DataFrame involving only the cast members of The Office
cast_df = pd.DataFrame()
cast_df['Cast Member'] = pd.Series(list(cast_set))
#cast_df = cast_df.set_index('Cast Member')

# Create a gender column for the cast members
cast_df['Gender'] = pd.Series(['M'] * len(cast_set))
female_list = ['Angela Kinsey', 'Phyllis Smith', 'Kate Flannery', 'Amy Adams', 'Mindy Kaling', 'Henriette Mantel',
              'Jenna Fischer', 'Melora Hardin']
cast_df.loc[cast_df['Cast Member'].isin(female_list), 'Gender'] = 'F'
cast_df

,Cast Member,Gender
0,B.J. Novak,M
1,Angela Kinsey,F
2,David Denman,M
3,Phyllis Smith,F
4,John Krasinski,M
5,Rainn Wilson,M
6,Mike McCaul,M
7,Brian Baumgartner,M
8,Kate Flannery,F
9,Craig Robinson,M
